# Project nightlight - Sending Telemetry & acting upon receiving commands

![Alt Text](https://media.giphy.com/media/Pnb5GTXdF54QxEaiLZ/giphy.gif) ![Alt Text](https://media.giphy.com/media/nINzArOjPdlhLYe6M9/giphy.gif)


## Código

In [1]:
import time
import json
import random
from counterfit_connection import CounterFitConnection
from counterfit_shims_grove.grove_light_sensor_v1_2 import GroveLightSensor
from counterfit_shims_grove.grove_led import GroveLed
import paho.mqtt.client as mqtt

# Inicializa a conexão simulada com o endereço IP e porta especificados
CounterFitConnection.init('127.0.0.1', 5001)

# Configura o sensor de luminosidade na porta 0
light_sensor = GroveLightSensor(0)

# Configura o LED na porta 5
led = GroveLed(5)

# Configuração do broker MQTT
broker = 'mqtt.eclipseprojects.io'  # Endereço do broker MQTT
port = 1883  # Porta padrão para conexões MQTT
telemetry_topic = "capacitacao-iot/telemetria"  # Tópico de telemetria para envio de dados
commands_topic = "capacitacao-iot/commandos" # tópico onde serão recebidos os comandos

# Gera um identificador único para o cliente MQTT
client_id = f'capacitacao-iot-mqtt-nightlight_client-{random.randint(0, 100000)}'
print("Client ID: " + client_id)

# Função de callback chamada ao conectar-se ao broker MQTT
def on_connect(client, userdata, flags, reason_code, properties):
    # Verifica se a conexão foi bem-sucedida ou houve falha
    if reason_code != 0:
        print(f"Falha ao conectar: {reason_code}. 'loop_forever()' tentará reconectar.")
    else:
        print("Conectado ao broker MQTT!")
        mqttc.subscribe(commands_topic)
        

# agora, o código do dispositivo também deverá reagir a eventos
# Por isso, ao receber uma mensagem, o dispositivo deve reagir ao que foi comandado. 
def handle_command(client, userdata, message):
    payload = json.loads(message.payload.decode())
    print("Command received:", payload)

    if payload['led_on']:
        led.on()
    else:
        led.off()

# Cria o cliente MQTT e configura a função de callback
mqttc = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2, client_id=client_id)
mqttc.on_connect = on_connect  # Associa a função de callback ao evento de conexão
mqttc.subscribe(commands_topic, qos=1)
mqttc.on_message = handle_command
mqttc.user_data_set([])  # Configura dados de usuário, se necessário

# Conecta ao broker MQTT e inicia o loop de comunicação
mqttc.connect(broker, port)
mqttc.loop_start()

# Aguarda um tempo inicial para garantir que a conexão esteja estabelecida
time.sleep(1)

# Loop principal para monitorar e enviar dados de luminosidade via MQTT
while True:
    # Lê o nível de luminosidade do sensor
    light = light_sensor.light

    # Formata a leitura do sensor em JSON para envio
    light_read = json.dumps({"light": light})

    # Envia os dados de luminosidade ao broker MQTT
    print("Enviando JSON:", light_read, "no tópico:", telemetry_topic)
    mqttc.publish(telemetry_topic, light_read)

    # Aguarda 1 segundo antes de repetir o ciclo
    time.sleep(3)


Client ID: capacitacao-iot-mqtt-nightlight_client-25728
Conectado ao broker MQTT!
Enviando JSON: {"light": 977} no tópico: capacitacao-iot/telemetria
Command received: {'led_on': False}
Command received: {'led_on': False}
Enviando JSON: {"light": 346} no tópico: capacitacao-iot/telemetria
Command received: {'led_on': False}
Command received: {'led_on': False}
Command received: {'led_on': False}
Command received: {'led_on': True}
Enviando JSON: {"light": 291} no tópico: capacitacao-iot/telemetria
Command received: {'led_on': True}
Command received: {'led_on': True}
Command received: {'led_on': False}
Command received: {'led_on': False}
Enviando JSON: {"light": 107} no tópico: capacitacao-iot/telemetria
Command received: {'led_on': False}
Command received: {'led_on': True}
Command received: {'led_on': False}
Command received: {'led_on': False}
Enviando JSON: {"light": 309} no tópico: capacitacao-iot/telemetria
Command received: {'led_on': False}
Command received: {'led_on': False}
Comman

KeyboardInterrupt: 

Command received: {'led_on': False}
Command received: {'led_on': False}
Command received: {'led_on': False}
Command received: {'led_on': False}
Command received: {'led_on': False}
Command received: {'led_on': True}
Command received: {'led_on': False}
Command received: {'led_on': False}


In [3]:
import time
from counterfit_connection import CounterFitConnection
from counterfit_shims_grove.grove_light_sensor_v1_2 import GroveLightSensor
from counterfit_shims_grove.grove_led import GroveLed
import json
import paho.mqtt.client as mqtt

CounterFitConnection.init('127.0.0.1', 5000)

light_sensor = GroveLightSensor(0)
led = GroveLed(5)

id = 'capacitacao_iot_'

client_telemetry_topic = id + '/telemetry'


# agora, o código do dispositivo também deverá reagir a eventos
# Por isso, deverá escutar por mensagens no tópico commands
server_command_topic = id + '/commands'
client_name = id + 'nightlight_client'

print(client_name)

mqtt_client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2,client_id=client_id)
#mqtt_client.connect('test.mosquitto.org')
#mqtt_client.connect('broker.mqttdashboard.com', 8000)
mqtt_client.connect('broker.emqx.io', 1883)


mqtt_client.loop_start()


# agora, o código do dispositivo também deverá reagir a eventos
# Por isso, ao receber uma mensagem, o dispositivo deve reagir ao que foi comandado. 
def handle_command(client, userdata, message):
    payload = json.loads(message.payload.decode())
    print("Command received:", payload)

    if payload['led_on']:
        led.on()
    else:
        led.off()

# agora, o código do dispositivo também deverá reagir a eventos
# Por isso, deve se inscrever no tópico /commands
mqtt_client.subscribe(server_command_topic, qos=1)
mqtt_client.on_message = handle_command

while True:
    light = light_sensor.light
    telemetry = json.dumps({'light' : light})
    
    print("Sending telemetry ", telemetry)

    mqtt_client.publish(client_telemetry_topic, telemetry)
    time.sleep(5)

capacitacao_iot_nightlight_client


NameError: name 'client_id' is not defined